In [7]:
import pandas as pd
import numpy as np
import collections
import gym
# !pip install tensorflow==1.15.0
import tensorflow as tf
tf.__version__
import tqdm

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple

from kaggle_environments import make, evaluate

from gym import spaces

import random

Loading environment football failed: No module named 'gfootball'


In [9]:
# df = pd.read_csv('age_range_video.csv')
# df

In [10]:
# df.sort_values('View rate', inplace=True)
# df.reset_index(inplace=True, drop=True)
# df

In [42]:
int(1) in range(0,100)

True

In [63]:
class MABanditGym:   
    
    
    def __init__(self, agent2="random"):
        ks_env = make("mab", debug=True)
        self.env = ks_env.train([None, agent2])
        self.nrounds = 2000
        self.banditCount = ks_env.configuration.banditCount
        self.prev_reward = 0
        
        # space 개념 : http://gym.openai.com/docs/#spaces
        self.action_space = spaces.Discrete(self.banditCount)
        low = -np.ones((self.nrounds,), dtype=np.float32)
        high = -low*(self.banditCount-1)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)
        # 최소 및 최대에 해당하는 reward 튜플
        self.reward_range = (-10,1) # 최소 -10, 최대 1
        self.grid = -np.ones((self.nrounds,2)) #2000,2 격자 state
        self.obs=np.array(self.grid).reshape(self.nrounds*2) #(4000,) 배열 생성
        # 안정된 기준선이 정의되지 않은 경우 오류 발생
        self.spec = None
        self.metadata = None
    def reset(self):
        #print(env.obs)
        self.env.reset()
        self.grid = -np.ones((self.nrounds,2))
        self.obs=np.array(self.grid).reshape(self.nrounds*2)
        self.prev_reward = 0
        return self.obs
    def change_reward(self, old_reward, done):
        if old_reward == 1000: # 에이전트가 이긴 경우
            return 0
        elif done: # 상대가 이긴 경우
            return -10
        else: # Reward 1/2000 
            return old_reward
    def step(self, action):
        _={}
        # Agent가 유효한 Action을 하는지 검증
        is_valid = (int(action) in range(0,self.banditCount)) #Bandit의 범위 안에 들어가는 Action일 경우 True 반환
        #valid_moves = [bnd for bnd in range(config.banditCount)]
       
        if is_valid: # True를 반환할 경우 Action 실행
            current_obs = self.env.step(int(action))
            
            for pos in range(0,2):
                #print(current_obs)
                self.grid[current_obs[0]['step']-1][pos]=current_obs[0]['lastActions'][pos]
            self.obs=np.array(self.grid).reshape(self.nrounds*2)
            old_reward= current_obs[0]['reward']
            done = (current_obs[0]['step']==self.nrounds-1 and current_obs[0]['reward']<600) #에피소드 끝나는 지점 설정
            reward = old_reward- self.prev_reward  #self.change_reward(old_reward, done)
            self.prev_reward=old_reward
        else: # False일 경우 게임 종료 후 Agent에게 벌칙 적용
            reward, done, _ = -10, True, {}
        #print(self.obs, reward, done, _  )    
        return self.obs, reward, done, _    


    
    def seed(self, seed=None): #시드값 고정
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]

# Create the environment for training tasks
env = MABanditGym(agent2="random")
# Set seed for experiment reproducibility
seed = 42
env.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()

In [64]:
#모델 구축
class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self, 
      num_actions: int, 
      num_hidden_units: int):
    """Initialize."""
    super().__init__()
    self.common = layers.Dense(num_hidden_units, activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = self.common(inputs)
    return self.actor(x), self.critic(x)

In [65]:
num_actions = env.action_space.n  # 100
print(num_actions)
num_hidden_units = 128

model = ActorCritic(num_actions, num_hidden_units)

100


In [66]:
random.betavariate(1, 5)

0.4088948191470329

## Training
 
1. 환경에서 에이전트를 실행하여 에피소드당 training 데이터를 수집합니다.
2. 각 단계에서 예측 return값을 계산합니다.
3. 모델 loss 계산
4. 역전파
5. 마지막 에피소드까지 1~4과정 반복

In [67]:
# Wrap OpenAI Gym's `env.step` call as an operation in a TensorFlow function.
# This would allow it to be included in a callable TensorFlow graph.

def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """Returns state, reward and done flag given an action."""

  state, reward, done, _ = env.step(action)
  return (state.astype(np.float32), 
          np.array(reward, np.int32), 
          np.array(done, np.int32))


def tf_env_step(action: tf.Tensor) -> List[tf.Tensor]:
  return tf.numpy_function(env_step, [action], 
                           [tf.float32, tf.int32, tf.int32])

In [68]:
tf.expand_dims([1,0], 0)

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[1, 0]])>

In [69]:
def run_episode(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> List[tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  initial_state_shape = initial_state.shape
  state = initial_state

  for t in tf.range(max_steps):
    # State를 Tensor 형태로 변환
    state = tf.expand_dims(state, 0)

    # 모델을 돌리고 Aciton 확률값과 Critic 값을 추출
    action_logits_t, value = model(state)

    # 다음 Sample Action에 Action 확률 적용
    action = tf.random.categorical(action_logits_t, 1)[0, 0]
    action_probs_t = tf.nn.softmax(action_logits_t)

    # Store critic values
    values = values.write(t, tf.squeeze(value))

    # 선택한 작업?의 로그 확률 저장
    action_probs = action_probs.write(t, action_probs_t[0, action])

    # 다음 State에 보상을 얻기 위한 Action 적용
    state, reward, done = tf_env_step(action)
    print(reward)
    state.set_shape(initial_state_shape)

     # Reward 저장
    rewards = rewards.write(t, reward)

    if tf.cast(done, tf.bool):
      break

  action_probs = action_probs.stack()
  values = values.stack()
  rewards = rewards.stack()

  return action_probs, values, rewards

In [70]:
def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float, 
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Rewards의 끝에서 부터 reward의 합계 저장 후 Returns에 저장
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward + gamma * discounted_sum
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) / 
               (tf.math.reduce_std(returns) + eps))

  return returns

In [71]:
huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

def compute_loss(
    action_probs: tf.Tensor,  
    values: tf.Tensor,  
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined actor-critic loss."""

  advantage = returns - values

  action_log_probs = tf.math.log(action_probs)
  actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

  critic_loss = huber_loss(values, returns)

  return actor_loss + critic_loss

In [72]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


@tf.function
def train_step(
    initial_state: tf.Tensor, 
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer, 
    gamma: float, 
    max_steps_per_episode: int) -> tf.Tensor:
  """Runs a model training step."""
  #print("Hello")
  with tf.GradientTape() as tape:

    # Run the model for one episode to collect training data
    action_probs, values, rewards = run_episode(
        initial_state, model, max_steps_per_episode) 

    # Calculate expected returns
    returns = get_expected_return(rewards, gamma)

    # Convert training data to appropriate TF tensor shapes
    action_probs, values, returns = [
        tf.expand_dims(x, 1) for x in [action_probs, values, returns]] 

    # Calculating loss values to update our network
    loss = compute_loss(action_probs, values, returns)

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  episode_reward = tf.math.reduce_sum(rewards)

  return episode_reward

In [73]:
%%time

max_episodes = 1
max_steps_per_episode = 2000

# 평균 reward가 600/100 이상이면 문제가 해결된 것으로 간주
# consecutive trials
reward_threshold = 600
running_reward = 0

# 감마(감가율) 설정
gamma = 0.99

with tqdm.trange(max_episodes) as t:
  for i in t:
    new_env=env.reset()
    #print(new_env)
    #print(type(new_env))
    #print(new_env.shape)
    
    initial_state = tf.constant(new_env, dtype=tf.float32)
    episode_reward = int(train_step(
        initial_state, model, optimizer, gamma, max_steps_per_episode))
    #print('episode_reward: ', episode_reward)
    running_reward = episode_reward*0.01 + running_reward*.99

    t.set_description(f'Episode {i}')
    t.set_postfix(
        episode_reward=episode_reward, running_reward=running_reward)

    # 에피소드 5회 당 평균 reward 확인
    if i % 5 == 0:
      print(f'Episode {i}: average reward: {running_reward}')

    if running_reward > reward_threshold:  
        break

print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')

  0%|          | 0/1 [00:00<?, ?it/s]

Tensor("while/PyFunc:1", dtype=int32, device=/job:localhost/replica:0/task:0)
Tensor("while/PyFunc:1", dtype=int32, device=/job:localhost/replica:0/task:0)


Episode 0: 100%|██████████| 1/1 [00:15<00:00, 15.19s/it, episode_reward=184, running_reward=1.84]

Episode 0: average reward: 1.84

Solved at episode 0: average reward: 1.84!
Wall time: 15.2 s


In [22]:
filename = 'bandit_model'
model.save(filename) # creates a HDF5 file 'bandit_model.h5'
#del model  # deletes the existing model

INFO:tensorflow:Assets written to: bandit_model\assets


In [23]:
from tensorflow.keras.models import load_model
# returns a compiled model
# identical to the previous one
model_bandit = load_model('bandit_model')

In [24]:
%%writefile agent_random_.py
import random
def agent_random_(obs, config):
    #print(obs)
    #print(config)
    valid_moves = [bnd for bnd in range(config['banditCount'])]
    return random.choice(valid_moves)

Writing agent_random_.py


In [25]:
%%writefile submission.py

import random
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model
# returns a compiled model
# identical to the previous one
model_bandit = load_model('bandit_model')


class MABanditPlayer:
    global model_bandit
    
    
    def __init__(self, observation, configuration):
        
        self.nrounds = configuration['episodeSteps']
        self.banditCount = configuration['banditCount']
        self.prev_reward = 0
        self.grid = -np.ones((self.nrounds,2))
        self.obs=np.array(self.grid).reshape(self.nrounds*2)
        self.prev_reward = 0
        
    def reset(self):
        #print(env.obs)
        self.env.reset()
        self.grid = -np.ones((self.nrounds,2))
        self.obs=np.array(self.grid).reshape(self.nrounds*2)
        self.prev_reward = 0
        return self.obs
    
    def play(self, observation, configuration):
        bandit=0
        if observation['step']>0:
            
        
            for pos in range(0,2):
                    #print(current_obs)
                    self.grid[observation['step']-1][pos]=observation['lastActions'][pos]
            new_reward= observation['reward']
            reward = new_reward- self.prev_reward
            self.prev_reward=new_reward
            self.obs=np.array(self.grid).reshape(self.nrounds*2)
            
            # Convert state into a batched tensor (batch size = 1)
            state = tf.expand_dims(self.obs,0)
            # Run the model and to get action probabilities and critic value
            action_logits_t, value = model_bandit(state)
            # Sample next action from the action probability distribution
            action = tf.random.categorical(action_logits_t, 1)[0, 0]
            #bandit = model_bandit.predict_classes(state) #state = tf.expand_dims(state, 0)
            with tf.compat.v1.Session() as sess:
                bandit = action.numpy()
            
        else:
            valid_moves = [bnd for bnd in range(configuration['banditCount'])]
            bandit = np.dtype('int32').type(random.choice(valid_moves))

        return(bandit)    
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
# Create the environment


# Set seed for experiment reproducibility
seed = 2021
#env.seed(seed)
random.seed(seed)
#tf.random.set_seed(seed)
#np.random.seed(seed)



observation0 = [{'remainingOverageTime': 60,
 'step': 0,
 'agentIndex': 0,
 'reward': 0,
 'lastActions': []}]
configuration0 = {'episodeSteps': 2000, 'actTimeout': 0.25, 'runTimeout': 1200, 'banditCount': 5, 'decayRate': 0.97, 'sampleResolution': 100}

mab_player = MABanditPlayer(observation0, configuration0)

def keras_agent(observation, configuration):
    #print(observation)
    #print(configuration)
    
    global mab_player
    bandit=0
    bandit=(mab_player.play(observation, configuration)).item()  
    return int(bandit)

Writing submission.py


In [26]:
env_test = make("mab", debug=True)

steps = env_test.run(["submission.py", "agent_random_.py"])

In [28]:
env_test.reset()
env_test.run(["submission.py", "agent_random_.py"])
env_test.render(mode='ipython', width=800, height=700)

KeyboardInterrupt: 

In [33]:
model